### 第一步
首先请运行[basic_analyze_process.sql](basic_analyze_process.sql)，进行基础查询,包括：<br>
1.  统计不同社区可见性状态的用户数量
2.  统计国籍分布情况
3.  计算不同国家玩家的平均游戏数量

### 第二步
使用pandas对[文件](../../data/processed/all_steam_and_game_data_after_cleaned.csv)进行更为细致的查询

In [277]:
import pandas as pd
import re
data = pd.read_csv('../../data/processed/all_steam_and_game_data_after_cleaned.csv')

In [278]:
data.columns[20]

'price_overview1'

统计游戏平均游玩时长

In [279]:
# 使用正则表达式提取playtime列以及对应的游戏时长和价格
selected_columns = []

for idx, col in enumerate(data.columns):
    if re.search(r'playtime', col):
        if idx <= 40:
            selected_columns.extend([data.columns[idx-1], data.columns[idx], data.columns[idx+2]])
        else:
            pass


# 创建包含这些列的新DataFrame
new_df = data[selected_columns]
new_df


,name_game1,playtime_forever_game1,price_overview1,name_game2,playtime_forever_game2,price_overview2,name_game3,playtime_forever_game3,price_overview3,name_game4,playtime_forever_game4,price_overview4,name_game5,playtime_forever_game5,price_overview5
0,Unknown,0,0,Unknown,0,0,Unknown,0,0,Unknown,0,0,Unknown,0,0
1,Unknown,0,0,Unknown,0,0,Unknown,0,0,Unknown,0,0,Unknown,0,0
2,Unknown,0,0,Unknown,0,0,Unknown,0,0,Unknown,0,0,Unknown,0,0
3,Counter-Strike2,371092,0,Counter-Strike,50721,9,DOOMEternal,4951,38,RiseoftheTombRaider,3903,29,Wreckfest,3742,29
4,Unknown,0,0,Unknown,0,0,Unknown,0,0,Unknown,0,0,Unknown,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20436,Unknown,0,0,Unknown,0,0,Unknown,0,0,Unknown,0,0,Unknown,0,0
20437,Unknown,0,0,Unknown,0,0,Unknown,0,0,Unknown,0,0,Unknown,0,0
20438,Unknown,0,0,Unknown,0,0,Unknown,0,0,Unknown,0,0,Unknown,0,0
20439,Unknown,0,0,Unknown,0,0,Unknown,0,0,Unknown,0,0,Unknown,0,0


In [280]:
games_df = pd.DataFrame()
new_df.rename(columns={col:''  for col in new_df.columns}, inplace=True) # 在进行concate时，要保证列名一致，因此此处清空列名

for i in range(0, len(new_df.columns), 3):   

    subset = new_df.iloc[:, i:i+3] # 选择当前组的三列
    
    # 如果games_df为空，直接赋值，否则在行方向上进行合并
    if games_df.empty:
        games_df = subset
    else:
        games_df = pd.concat([games_df, subset], axis=0,ignore_index=True ) 

games_df.columns=['game_name','playtime_forever','game_price']
games_df.to_csv('./query_results/most_frequent_play_game_all_timelist.csv')       

C:\Users\55386\AppData\Local\Temp\ipykernel_1804\2834459858.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.rename(columns={col:''  for col in new_df.columns}, inplace=True) # 在进行concate时，要保证列名一致，因此此处清空列名


In [281]:
games_df = games_df[games_df['game_name']!=('Unknown')]
games_df

,game_name,playtime_forever,game_price
3,Counter-Strike2,371092,0
7,Counter-Strike2,424976,0
10,RocketLeague,37642,0
11,DOOMEternal,14908,38
13,Rust,104399,39
...,...,...,...
102143,GWENT:TheWitcherCardGame,10743,0
102152,AdVentureCapitalist,7540,0
102157,Rust,20124,39
102159,PUBG:BATTLEGROUNDS,1215,0


In [282]:
grouped = games_df.groupby('game_name').agg({
    'playtime_forever': ['mean', 'median','count']
})

# 展示结果
grouped

playtime_forever               
                                 mean   median count
game_name                                           
100%OrangeJuice          31540.666667   8249.5     6
11-11MemoriesRetold          0.000000      0.0     2
12LaboursofHercules          0.000000      0.0     1
1v1.LOL                    342.500000    342.5     2
3DCoatModdingTool        14872.000000  14872.0     1
...                               ...      ...   ...
太吾绘卷TheScrollOfTaiwu     11926.000000  11926.0     1
我来自江湖                     1000.000000   1000.0     1
暖雪WarmSnow                5762.000000   5762.0     1
雀魂麻将(MahjongSoul)         3341.800000   3052.0     5
鬼谷八荒TaleofImmortal       14674.200000  16642.0     5

[2267 rows x 3 columns]

In [283]:
# 将多级列名展平成单级列名
grouped.columns = ['_'.join(col) for col in grouped.columns]

# 重置索引，使其变为平面格式
grouped.reset_index(inplace=True)
grouped

,game_name,playtime_forever_mean,playtime_forever_median,playtime_forever_count
0,100%OrangeJuice,31540.666667,8249.5,6
1,11-11MemoriesRetold,0.000000,0.0,2
2,12LaboursofHercules,0.000000,0.0,1
3,1v1.LOL,342.500000,342.5,2
4,3DCoatModdingTool,14872.000000,14872.0,1
...,...,...,...,...
2262,太吾绘卷TheScrollOfTaiwu,11926.000000,11926.0,1
2263,我来自江湖,1000.000000,1000.0,1
2264,暖雪WarmSnow,5762.000000,5762.0,1
2265,雀魂麻将(MahjongSoul),3341.800000,3052.0,5


[](./query_results/)